<a href="https://colab.research.google.com/github/kjh1311990/face/blob/main/Fuzzy_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
from google.colab import drive
drive.mount('/content/drive')

!pip install komm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
import komm
import numpy as np
code = komm.BCHCode(11,365)

In [130]:
import numpy as np

PARAMS_M = 2047
PARAMS_N = 512

PARAMS_LOGQ = 9
PARAMS_LOGL = 5
PARAMS_LOGT = 2

PARAMS_Q = 1 << PARAMS_LOGQ
PARAMS_L = 1 << PARAMS_LOGL

In [131]:
def EMBLEM_Encode(m, shift):
    encm = ((m << 1) + 1) << (shift - 1);
    return encm.reshape(PARAMS_M, 1);

def EMBLEM_Decode(m, shift):
    decm = m >> shift;
    return decm.reshape(PARAMS_M, 1);

In [132]:
Align_Image();

In [133]:
from keras.models import load_model
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import dlib, cv2

def Setup():
  A = np.random.randint(low = -PARAMS_L/2, high = PARAMS_L/2+1, size = PARAMS_N * PARAMS_M, dtype = int)
  A.resize(PARAMS_M, PARAMS_N)

  model = load_model('/content/drive/MyDrive/Notebooks/weights.h5')
  model2 = Model(model.input, model.layers[-2].output)

  return (A, model2);

def Encap(A, model, folder):
  img = np.zeros((8,224,224,3));
  feature = np.zeros((8,512));

  for i in range(0,8):
    
    detector = dlib.get_frontal_face_detector();
    sp = dlib.shape_predictor('/content/drive/MyDrive/Notebooks/shape_predictor_5_face_landmarks.dat')
    tmp = dlib.load_rgb_image(folder + '/' + str(i) +'.jpg')
    tmp_detection = detector(tmp, 1)
    tmp_shape = sp(tmp, tmp_detection[0])
    tmp_aligned = dlib.get_face_chip(tmp, tmp_shape)
    tmp_aligned = cv2.cvtColor(tmp_aligned, cv2.COLOR_BGR2RGB)
    cv2.imwrite('tmp.jpg', tmp_aligned)

    img[i] = image.load_img('tmp.jpg', target_size=(224, 224))
    x = image.img_to_array(img[i]);
    x = np.expand_dims(x, axis=0);
    x = preprocess_input(x);
    feature[i] = model.predict(x);
    !rm tmp.jpg
  t = (feature.max(axis=0) - feature.min(axis=0));

  for i in range(0,8):
    for j in range(0,512):
      if t[j] > 1:
        feature[i][j] = (feature[i][j] / (6* t[j]))

  feature = feature.astype(int);

  b = feature[0].reshape(PARAMS_N,1);

  m = np.random.randint(low = 0, high = 2, size = 133, dtype = int);

  encm = code.encode(m);
  encm.reshape(2047,1);

  c = np.remainder(np.matmul(A,b) + EMBLEM_Encode(encm, PARAMS_LOGQ-1), PARAMS_Q);


  c = (c + (1 << (PARAMS_LOGQ - PARAMS_LOGT - 1))) >> (PARAMS_LOGQ - PARAMS_LOGT);

  return (t, c, m)

def Decap(A, model, t, c, imgg):

  detector = dlib.get_frontal_face_detector();
  sp = dlib.shape_predictor('/content/drive/MyDrive/Notebooks/shape_predictor_5_face_landmarks.dat')
  tmp = dlib.load_rgb_image(imgg)
  tmp_detection = detector(tmp, 1)
  tmp_shape = sp(tmp, tmp_detection[0])
  tmp_aligned = dlib.get_face_chip(tmp, tmp_shape)
  tmp_aligned = cv2.cvtColor(tmp_aligned, cv2.COLOR_BGR2RGB)
  cv2.imwrite('tmp.jpg', tmp_aligned)

  img = image.load_img('tmp.jpg', target_size=(224, 224))
  !rm tmp.jpg
  
  x = image.img_to_array(img);
  x = np.expand_dims(x, axis=0);
  x = preprocess_input(x);
  feature = model.predict(x);
  feature = feature.reshape(PARAMS_N,1);

  for j in range(0,512):
    if t[j] > 1:
      feature[j] = (feature[j] / (6* t[j]))

  bprime = feature.astype(int);

  c = c << (PARAMS_LOGQ - PARAMS_LOGT);
  cprime = np.remainder(c - np.matmul(A, bprime), PARAMS_Q)
  decm = EMBLEM_Decode(cprime, PARAMS_LOGQ-1).reshape(2047)
  mprime = code.decode(decm);
  return mprime;

In [134]:
(A, model) = Setup();

In [135]:
(t, c, m) = Encap(A, model, '/content/drive/MyDrive/Notebooks/face/input/1');

In [136]:
mprime = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/1/1.jpg');

In [137]:
np.sum(np.abs(m - mprime))

0

In [138]:
mprime2 = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/2/1.jpg');

In [139]:
np.sum(np.abs(m - mprime2))

70

In [140]:
mprime3 = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/1/8.jpg');

In [141]:
np.sum(np.abs(m - mprime3))

0

In [158]:
for i in range(0,6):
  mprime3 = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/2/'+ str(i) + '.jpg');
  print(np.sum(np.abs(m - mprime3)))

62
70
76
65
64
66


In [159]:
for i in range(0,10):
  mprime3 = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/3/'+ str(i) + '.jpg');
  print(np.sum(np.abs(m - mprime3)))

60
69
64
74
71
82
58
65
68
72


In [160]:
for i in range(0,16):
  mprime3 = Decap(A, model, t, c, '/content/drive/MyDrive/Notebooks/face/input/1/'+ str(i) + '.jpg');
  print(np.sum(np.abs(m - mprime3)))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
